# Imersão Dados - Alura

## Aula 5 - Validação de modelo e _Overfit_

Como foi visto na aula anterior, os modelos de _machine learning_ possuem uma dependência de fatores "aleatórios" que podem ser introduzidos desde as primeiras etapas do processo de criação de um modelo. Então vamos discutir formas de reduzir essa aleatoriedade e validar nosso modelo.

### Índice

- [Diminuindo efeitos de aleatoriedade: _Cross-validation_](#_cross-validation_)
- [Embaralhando os dados](#embaralhando-os-dados)
- [_Overfit_](#_overfit_)

### _Cross-validation_

A primeira coisa a ser feita é utilizar um modelo que possua menos dependência do parâmetro `random_state` e o _decision tree_ atende esse requisito.

In [ ]:
# carrega os dados e separa
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

dados = pd.read_csv('https://github.com/alura-cursos/imersao-dados-2-2020/blob/master/MICRODADOS_ENEM_2019_SAMPLE_43278.csv?raw=true')
dados = dados[['NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_CN', 'NU_NOTA_REDACAO', 'NU_NOTA_MT']].dropna()
x = dados[['NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_CN', 'NU_NOTA_REDACAO']]
y = dados['NU_NOTA_MT']

from sklearn.model_selection import train_test_split
SEED = 1234
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.25, random_state=SEED)

In [ ]:
# roda modelo para conferir
from sklearn.svm import LinearSVR
modelo = LinearSVR(random_state=SEED)
modelo.fit(x_treino, y_treino)
predicao = novo_modelo.predict(x_teste)

from sklearn.metrics import mean_squared_error
mean_squared_error(y_teste, predicao)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


12745.739089201345

In [ ]:
# agora sim vai para o modelo arvore decisão
from sklearn.model_selection import train_test_split

modelo_arvore = DecisionTreeRegressor(max_depth=3)
modelo_arvore.fit(x_treino, y_treino)
predicao_arvore = modelo_arvore.predict(x_teste)
mean_squared_error(y_teste, predicao_arvore)

5987.352568520455

Mas ainda temos fator de aleatoriedade que vem do processo de divisão dos dados e para mitigar esse efeito podemos utilizar o _cross-validation method_, que possui algumas variações mas vamos nos concentrar em apenas uma forma de aplicá-lo.

Seguindo esse método, ao invés de separarmos nosso _DataSet_ em apenas dois grupos (treino e teste), vamos separá-lo em cinco grupos, por exemplo. Com os grupos separados vamos executar N rodadas de treino com os quatro primeiros e depois testar com o quinto. Após isso trocamos o grupo testado por um dos que utilizados no treinamento, e assim por diante até completar a rotação dos grupos. Dessa forma o resultado obtido será uma média de "todos os modelos".

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_validate

modelo_arvore = DecisionTreeRegressor(max_depth=3)
cross_validate(modelo_arvore, x, y)

{'fit_time': array([0.093436  , 0.08555746, 0.08088923, 0.07915139, 0.08505177]),
 'score_time': array([0.00366378, 0.00273538, 0.00241041, 0.0025568 , 0.00253868]),
 'test_score': array([0.48262421, 0.51140553, 0.48905884, 0.5252888 , 0.44483146])}

O método `cross_validate` faz todas as etapas: a separação dos dados (cinco grupos por padrão) e as rodadas de treinamento e teste.

E perceba que os valores de `test_score` parecem estar em uma escala diferente da que estavamos trabalhando, isso é porque a métrica padrão utlizada para avaliar o erro do modelo é uma grandeza diferente da que estávamos usando, então vamos especificar a grandeza que queremos e também a quantidade de grupos que os dados devem ser divididos:

In [ ]:
resultado_cv = cross_validate(modelo_arvore, x, y, scoring='neg_mean_squared_error', cv=10)
resultado_cv

{'fit_time': array([0.10209703, 0.09092641, 0.08909249, 0.09380293, 0.08891463,
        0.0875752 , 0.08738399, 0.08620048, 0.08725715, 0.08907795]),
 'score_time': array([0.00176954, 0.00173926, 0.00162387, 0.00164247, 0.00166059,
        0.0016222 , 0.00170636, 0.00179124, 0.00198793, 0.00165391]),
 'test_score': array([-5686.95455159, -6035.40144634, -5830.55015277, -5958.32070827,
        -5749.551073  , -6292.77890429, -6161.57457333, -6222.41251527,
        -6141.63385177, -6603.98813674])}

Agora note que os valores de erro estão mais parecidos com o que haviamos determinado, mas com sinal negativo. Isso ocorre devido a uma regra de desenvolvimento do `scikit-learn` de que "quanto maior o valor melhor". No caso do ERQ "quanto mais próximo de zero melhor", então tratando o erro como um valor negativo, a regra "quanto maior melhor" é satisfeita.

Como não vamos utilizar outros processos de otimização que usam a mesma regra, podemos passar o ERQ para positivo e tirar a média de uma vez. Além disso podemos determinar o desvio padrão dos ERQs e determinar um intervalo de confiança:

In [ ]:
media_cv = (resultado_cv['test_score']*-1).mean()
media_cv

6068.3165913384655

In [ ]:
desvio_cv = (resultado_cv['test_score']*-1).std()
desvio_cv

263.04166999750663

In [ ]:
lim_inferior = media_cv - (2*desvio_cv)
lim_superior = media_cv + (2*desvio_cv)
print(f"Intervalo de confiança: {lim_inferior} - {lim_superior}")

Intervalo de confiança: 5542.233251343452 - 6594.399931333479


In [ ]:
# função para calculo do intervalo de confiança
def calcula_erro(resultados_df):
    media = (resultados_df['test_score']*-1).mean()
    desvio = (resultados_df['test_score']*-1).std()
    lim_inferior = media - (2*desvio)
    lim_superior = media + (2*desvio)
    print(f"Intervalo de confiança: {lim_inferior} - {lim_superior}")

### Embaralhando os dados

Se verificarmos a documentação do método [`cross_validate`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html) vamos perceber que ele não tem um parâmetro de aleatoriedade e a forma como a divisão dos dados é feita é bem definida. Isso significa que se o _DataSet_ estiver organizado em ordem crescente de acordo com alguma das notas, por exemplo, podemos ter um modelo com resultados enviesados.

Podemos embaralhar os dados usando um método da biblioteca `scikit-learn`:

In [ ]:
from sklearn.model_selection import KFold

partes = KFold(n_splits=10, shuffle=True)

calcula_erro(cross_validate(modelo_arvore, x, y, scoring='neg_mean_squared_error', cv=partes))

Intervalo de confiança: 5824.493018231576 - 6298.992449847258


E também podemos definir um _random state_, basta executar um método da biblioteca `numpy` na célula e as operações passam a ser executadas com esse fator de aleatoriedade (?).

In [ ]:
# sem fator aleatorio definido
calcula_erro(cross_validate(modelo_arvore, x, y, scoring='neg_mean_squared_error', cv=partes))

Intervalo de confiança: 5820.353373452573 - 6297.575473759863


In [ ]:
# definindo fator aleatorio
import numpy as np

np.random.seed(SEED)

calcula_erro(cross_validate(modelo_arvore, x, y, scoring='neg_mean_squared_error', cv=partes))

Intervalo de confiança: 5910.513672291329 - 6210.679992504457


In [ ]:
# executando novamente com o mesmo fator de aleatoriedade
np.random.seed(SEED)
calcula_erro(cross_validate(modelo_arvore, x, y, scoring='neg_mean_squared_error', cv=partes))

Intervalo de confiança: 5910.513672291329 - 6210.679992504457


### _Overfit_

In [ ]:
def regressor_arvore(nivel_mais_profundo):
    valor_maximo = 20
    nivel_mais_profundo = valor_maximo if (nivel_mais_profundo < 1 or nivel_mais_profundo > valor_maximo) else nivel_mais_profundo

    print("Nível\tMédia (ERQ)\t\tDesvio padrão")
    for nivel in range(1, nivel_mais_profundo+1):
        np.random.seed(SEED)
        partes = KFold(n_splits=10, shuffle=True)
        modelo_arvore = DecisionTreeRegressor(max_depth=nivel)
        resultados = cross_validate(modelo_arvore, x, y, scoring='neg_mean_squared_error', cv=partes)

        media = (resultados['test_score']*-1).mean()
        desvio = (resultados['test_score']*-1).std()
        lim_inferior = media - (2*desvio)
        lim_superior = media + (2*desvio)
        
        # print(f"{nivel}\t{media}\t{lim_superior - lim_inferior}")
        print(f"{nivel}\t{media}\t{desvio}")

In [ ]:
regressor_arvore(20)

Nível	|Média (ERQ)		|Desvio padrão
1	7866.927011683838	65.33626494272087
2	6558.321643161868	75.92664374939238
3	6060.596832397893	75.0415800532818
4	5831.914600536327	73.63692970409825
5	5691.016497534935	79.58409872098808
6	5591.73672389537	71.7875095008164
7	5536.431104799163	62.21478643073848
8	5542.102109416983	72.89547409382575
9	5591.8419392935775	71.29206246665687
10	5731.32564405446	65.79248497846966
11	5915.804789201808	80.93533670789705
12	6179.0090814897785	81.7524210056854
13	6505.0166634314355	111.04682238290856
14	6828.826593401468	113.15901341339163
15	7199.633672703089	92.8101440766405
16	7573.52893043737	113.64179827624405
17	7959.784852838859	134.0793247637921
18	8357.188510330792	131.9068585877019
19	8767.980901546043	130.12327404153444
20	9122.313721122862	123.63080258328979


Podemos notar que conforme o nível vai aumentando o ERQ vai diminuindo, porém esse comportamento ocorre até um certo nível. Neste caso a partir do nível 8 o ERQ passa a aumentar.

Vamos passar o parâmetro `return_train_score` como `True` ao chamar o método `cross_validate` para que o _score_ dos dados de treino também sejam retornados:

In [ ]:
def regressor_arvore(nivel_mais_profundo):
    valor_maximo = 20
    nivel_mais_profundo = valor_maximo if (nivel_mais_profundo < 1 or nivel_mais_profundo > valor_maximo) else nivel_mais_profundo

    print("Profundidade\t| Média (treino)\t| Desvio padrão\t\t| Média (teste)\t\t| Desvio padrão\t")
    print("----------------|-----------------------|-----------------------|-----------------------|------------------")
    for nivel in range(1, nivel_mais_profundo+1):
        np.random.seed(SEED)
        partes = KFold(n_splits=10, shuffle=True)
        modelo_arvore = DecisionTreeRegressor(max_depth=nivel)

        resultados = cross_validate(modelo_arvore, x, y, scoring='neg_mean_squared_error', cv=partes, return_train_score=True)

        media_treino = (resultados['train_score']*-1).mean()
        desvio_treino = (resultados['train_score']*-1).std()
        media_teste = (resultados['test_score']*-1).mean()
        desvio_teste = (resultados['test_score']*-1).std()
        
        print(f"{nivel}\t\t|{media_treino}\t|{desvio_treino}\t|{media_teste}\t|{desvio_teste}")

In [ ]:
regressor_arvore(20)

Profundidade	| Média (treino)	| Desvio padrão		| Média (teste)		| Desvio padrão	
----------------|-----------------------|-----------------------|-----------------------|------------------
1		|7849.445766058474	|7.545010440052587	|7866.927011683838	|65.33626494272087
2		|6532.7483836249085	|8.237688160023687	|6558.321643161868	|75.92664374939238
3		|6025.718727804219	|8.25577596657425	|6060.596832397893	|75.0415800532818
4		|5766.615419484886	|9.343600244926202	|5831.914600536327	|73.63692970409825
5		|5604.129644787683	|10.004259579957926	|5691.016497534935	|79.58409872098808
6		|5470.794376514847	|9.862625980188787	|5591.73672389537	|71.7875095008164
7		|5368.22113220513	|8.42960342995329	|5536.431104799163	|62.21478643073848
8		|5275.114664891669	|9.782089735034074	|5542.102109416983	|72.89547409382575
9		|5166.478005291161	|11.07054312283756	|5591.8419392935775	|71.29206246665687
10		|5023.719373671476	|12.278287771867753	|5731.32564405446	|65.79248497846966
11		|4835.375975792986	

Ao contrário do que foi observado para os para os dados de teste, para os dados de treino verificamos que não há um "ponto de retorno". Para estes, a medida que a profundidade aumenta temos um ERQ cada vez menor.

O que acontece é que o modelo está ficando muito bom com os dados de treino,como se estivesse praticamente decorando eles, esse comportamento é chamado de _overfit_.

In [ ]:
# pesquisar sobre intervalo de confiança
# testar outros parâmetros do DecisionTreeRegressor()
# procurar outras formas de realizar os ajustes de parâmetros com o sklearn
# pesquisar o que é o problema de underfit
# plotar gráficos de 'test_score x train_score' e '*_score x `parâmetro do modelo`'